In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import seaborn as sns
import holidays
from sklearn.linear_model import LinearRegression
from sklearn.metrics import (explained_variance_score, 
                             mean_absolute_error, 
                             mean_squared_error, 
                             mean_squared_log_error,
                             r2_score)
from sklearn.model_selection import (GridSearchCV,
                                     TimeSeriesSplit)
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline


from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
import matplotlib.pyplot as plt
import time
import seaborn as sns
import holidays
from sklearn.linear_model import LinearRegression
from sklearn.metrics import (explained_variance_score, 
                             mean_absolute_error, 
                             mean_squared_error, 
                             mean_squared_log_error,
                             r2_score)
from sklearn.model_selection import (GridSearchCV,
                                     TimeSeriesSplit)
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline


from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RandomizedSearchCV

import xgboost as xgb
from xgboost import plot_importance, plot_tree

c:\Users\augusto.lourencatto\Desktop\me\TCC\venv\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [5]:
df : pd.DataFrame = pd.read_csv('./data/checkpoints/df_demanda_potencia_v2.csv',index_col='Unnamed: 0')
df.head(2)


,precipitacao_total,pressao_atmosferica,radiacao_global,temperatura_ar,temperatura_ponto_carvalho,umidade_relativa_ar,vento_direcao,velocidade_vento,id_subsistema,nom_subsistema,...,DayOfYear,WeekOfYear,lag_1,lag_7,lag_14,rolling_mean_1,rolling_mean_7,rolling_mean_14,el_nino,la_nina
2012-01-01 00:00:00,0.88,947.22,-2.8518,21.70,19.7,90.25,95.8,1.48,SE,SUDESTE,...,1,52,NaN,NaN,NaN,NaN,NaN,NaN,0,0
2012-01-01 01:00:00,2.00,947.30,-2.4346,21.24,19.5,92.00,114.0,0.80,SE,SUDESTE,...,1,52,NaN,NaN,NaN,NaN,NaN,NaN,0,0


In [27]:
# Interpolando com o metodo de bfill
#TODO: Ver se é o melhor método
#https://note.nkmk.me/en/python-pandas-interpolate/#:~:text=You%20can%20interpolate%20missing%20values,and%20Series%20with%20interpolate()%20.&text=Use%20dropna()%20and%20fillna,them%20with%20a%20specific%20value.
df : pd.DataFrame = df.fillna(method='bfill')

In [29]:
df['val_cargaenergiahomwmed'].isnull().sum()

# df['val_cargaenergiahomwmed'][df['val_cargaenergiahomwmed'].isnull()]

0

In [15]:
# Dividindo os dados de treino e teste

df_train_xg = df.loc["2012-01-01":"2020-31-12 00:00"]
df_test_xg = df.loc["2021-01-01 00:00":"2021-12-31 23:00"]

columns_to_drop : list = ["val_cargaenergiahomwmed","nom_subsistema","id_subsistema","dataref","max_executado","DATE","ORDINAL_DATE","LINREG","dia_semana"]

print(df_train_xg.shape,df_test_xg.shape)

X_train_xg = df_train_xg.drop(columns=columns_to_drop)
y_train_xg = df_train_xg["val_cargaenergiahomwmed"]
X_test_xg = df_test_xg.drop(columns=columns_to_drop)
y_test_xg = df_test_xg["val_cargaenergiahomwmed"]

(78910, 35) (8759, 35)


In [16]:
X_train_xg.head()
X_train_xg.to_csv("./data/temp/X_train_xg.csv")

In [20]:
cv = TimeSeriesSplit(n_splits=5)

parameters = {"learning_rate": [0.1, 0.05],
              #The learning rate is the shrinkage you do at every step you are making

               "gamma" : [0.01, 0.1, 0.3, 0.5],
              #A node is split only when the resulting split gives a positive reduction in the loss function. Gamma specifies the minimum loss reduction required to make a split.
              #Makes the algorithm conservative. The values can vary depending on the loss function and should be tuned.
              
               "max_depth": [2, 4, 7, 10],
              #The maximum depth of a tree, same as GBM.
              #Used to control over-fitting as higher depth will allow model to learn relations very specific to a particular sample.

               "colsample_bytree": [0.3, 0.6],
              #Denotes the fraction of columns to be randomly samples for each tree.

               "subsample": [0.2, 0.4, 0.5],
              #Denotes the fraction of observations to be randomly samples for each tree.
              #Lower values make the algorithm more conservative and prevents overfitting but too small values might lead to under-fitting.

               "reg_alpha": [0, 0.5, 1],
              #L1 regularization term on weights. Increasing this value will make model more conservative.

               "reg_lambda": [1, 1.5, 2, 3, 4.5],
              #L2 regularization term on weights. Increasing this value will make model more conservative.

               "min_child_weight": [1, 3, 5, 7],
              #The larger min_child_weight is, the more conservative the algorithm will be.

               "n_estimators": [100, 250, 500]}
              #Number of tress or rounds. The more, the higher the risk of overfitting

xgbr = xgb.XGBRegressor(objective='reg:squarederror' , sampling_method = 'gradient_based', tree_method = 'gpu_hist')

xgb_rscv = RandomizedSearchCV(xgbr, param_distributions = parameters,  verbose = 3, random_state = 40, cv=cv)

In [21]:
# Label containes NaN

model_xgboost = xgb_rscv.fit(X_train_xg, y_train_xg)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


c:\Users\augusto.lourencatto\Desktop\me\TCC\venv\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
c:\Users\augusto.lourencatto\Desktop\me\TCC\venv\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\augusto.lourencatto\Desktop\me\TCC\venv\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "c:\Users\augusto.lourencatto\Desktop\me\TCC\venv\lib\site-packages\sklearn\metrics\_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "c:\Users\augusto.lourencatto\Desktop\me\TCC\venv\lib\site

[CV 1/5] END colsample_bytree=0.3, gamma=0.3, learning_rate=0.05, max_depth=4, min_child_weight=3, n_estimators=500, reg_alpha=0.5, reg_lambda=3, subsample=0.4;, score=nan total time=   3.3s
[CV 2/5] END colsample_bytree=0.3, gamma=0.3, learning_rate=0.05, max_depth=4, min_child_weight=3, n_estimators=500, reg_alpha=0.5, reg_lambda=3, subsample=0.4;, score=nan total time=   0.0s
[CV 3/5] END colsample_bytree=0.3, gamma=0.3, learning_rate=0.05, max_depth=4, min_child_weight=3, n_estimators=500, reg_alpha=0.5, reg_lambda=3, subsample=0.4;, score=nan total time=   0.0s
[CV 4/5] END colsample_bytree=0.3, gamma=0.3, learning_rate=0.05, max_depth=4, min_child_weight=3, n_estimators=500, reg_alpha=0.5, reg_lambda=3, subsample=0.4;, score=nan total time=   0.0s
[CV 5/5] END colsample_bytree=0.3, gamma=0.3, learning_rate=0.05, max_depth=4, min_child_weight=3, n_estimators=500, reg_alpha=0.5, reg_lambda=3, subsample=0.4;, score=nan total time=   0.0s


c:\Users\augusto.lourencatto\Desktop\me\TCC\venv\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\augusto.lourencatto\Desktop\me\TCC\venv\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "c:\Users\augusto.lourencatto\Desktop\me\TCC\venv\lib\site-packages\sklearn\metrics\_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "c:\Users\augusto.lourencatto\Desktop\me\TCC\venv\lib\site-packages\sklearn\base.py", line 706, in score
    return r2_score(y, y_pred, sample_weight=sample_weight)
  File "c:\Users\augusto.lourencatto\Desktop\me\TCC\venv\lib\site-packages\sklearn\metrics\_regression.py", line 789, in r2_score
    y_type, y_true, y_pred, multioutput = _check_reg_targets(
  File 

[CV 1/5] END colsample_bytree=0.6, gamma=0.5, learning_rate=0.1, max_depth=2, min_child_weight=7, n_estimators=250, reg_alpha=1, reg_lambda=2, subsample=0.4;, score=nan total time=   0.7s
[CV 2/5] END colsample_bytree=0.6, gamma=0.5, learning_rate=0.1, max_depth=2, min_child_weight=7, n_estimators=250, reg_alpha=1, reg_lambda=2, subsample=0.4;, score=nan total time=   0.0s
[CV 3/5] END colsample_bytree=0.6, gamma=0.5, learning_rate=0.1, max_depth=2, min_child_weight=7, n_estimators=250, reg_alpha=1, reg_lambda=2, subsample=0.4;, score=nan total time=   0.0s
[CV 4/5] END colsample_bytree=0.6, gamma=0.5, learning_rate=0.1, max_depth=2, min_child_weight=7, n_estimators=250, reg_alpha=1, reg_lambda=2, subsample=0.4;, score=nan total time=   0.0s
[CV 5/5] END colsample_bytree=0.6, gamma=0.5, learning_rate=0.1, max_depth=2, min_child_weight=7, n_estimators=250, reg_alpha=1, reg_lambda=2, subsample=0.4;, score=nan total time=   0.0s


...

c:\Users\augusto.lourencatto\Desktop\me\TCC\venv\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\augusto.lourencatto\Desktop\me\TCC\venv\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "c:\Users\augusto.lourencatto\Desktop\me\TCC\venv\lib\site-packages\sklearn\metrics\_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "c:\Users\augusto.lourencatto\Desktop\me\TCC\venv\lib\site-packages\sklearn\base.py", line 706, in score
    return r2_score(y, y_pred, sample_weight=sample_weight)
  File "c:\Users\augusto.lourencatto\Desktop\me\TCC\venv\lib\site-packages\sklearn\metrics\_regression.py", line 789, in r2_score
    y_type, y_true, y_pred, multioutput = _check_reg_targets(
  File 

[CV 1/5] END colsample_bytree=0.3, gamma=0.1, learning_rate=0.05, max_depth=7, min_child_weight=5, n_estimators=100, reg_alpha=0, reg_lambda=3, subsample=0.2;, score=nan total time=   4.6s
[CV 2/5] END colsample_bytree=0.3, gamma=0.1, learning_rate=0.05, max_depth=7, min_child_weight=5, n_estimators=100, reg_alpha=0, reg_lambda=3, subsample=0.2;, score=nan total time=   0.0s
[CV 3/5] END colsample_bytree=0.3, gamma=0.1, learning_rate=0.05, max_depth=7, min_child_weight=5, n_estimators=100, reg_alpha=0, reg_lambda=3, subsample=0.2;, score=nan total time=   0.0s
[CV 4/5] END colsample_bytree=0.3, gamma=0.1, learning_rate=0.05, max_depth=7, min_child_weight=5, n_estimators=100, reg_alpha=0, reg_lambda=3, subsample=0.2;, score=nan total time=   0.0s
[CV 5/5] END colsample_bytree=0.3, gamma=0.1, learning_rate=0.05, max_depth=7, min_child_weight=5, n_estimators=100, reg_alpha=0, reg_lambda=3, subsample=0.2;, score=nan total time=   0.0s


c:\Users\augusto.lourencatto\Desktop\me\TCC\venv\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
c:\Users\augusto.lourencatto\Desktop\me\TCC\venv\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\augusto.lourencatto\Desktop\me\TCC\venv\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "c:\Users\augusto.lourencatto\Desktop\me\TCC\venv\lib\site-packages\sklearn\metrics\_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "c:\Users\augusto.lourencatto\Desktop\me\TCC\venv\lib\site

[CV 1/5] END colsample_bytree=0.3, gamma=0.3, learning_rate=0.1, max_depth=10, min_child_weight=1, n_estimators=100, reg_alpha=1, reg_lambda=1.5, subsample=0.5;, score=nan total time=  18.7s
[CV 2/5] END colsample_bytree=0.3, gamma=0.3, learning_rate=0.1, max_depth=10, min_child_weight=1, n_estimators=100, reg_alpha=1, reg_lambda=1.5, subsample=0.5;, score=nan total time=   0.0s
[CV 3/5] END colsample_bytree=0.3, gamma=0.3, learning_rate=0.1, max_depth=10, min_child_weight=1, n_estimators=100, reg_alpha=1, reg_lambda=1.5, subsample=0.5;, score=nan total time=   0.0s
[CV 4/5] END colsample_bytree=0.3, gamma=0.3, learning_rate=0.1, max_depth=10, min_child_weight=1, n_estimators=100, reg_alpha=1, reg_lambda=1.5, subsample=0.5;, score=nan total time=   0.0s
[CV 5/5] END colsample_bytree=0.3, gamma=0.3, learning_rate=0.1, max_depth=10, min_child_weight=1, n_estimators=100, reg_alpha=1, reg_lambda=1.5, subsample=0.5;, score=nan total time=   0.0s


XGBoostError: [11:09:31] C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/data/data.cc:367: Check failed: valid: Label contains NaN, infinity or a value too large.